In [ ]:
import numpy as np
import random
from concurrent.futures import ThreadPoolExecutor

def rastrigin_function(x):
    """
    Rastrigin function for optimization.
    x: List or array of genes (real numbers)
    """
    A = 10  # Constant
    return A * len(x) + sum([(xi ** 2) - A * np.cos(2 * np.pi * xi) for xi in x])

def initialize_population(grid_size, lower_bound, upper_bound):
    """
    Initialize the grid with random solutions (cells).
    grid_size: Tuple representing the grid dimensions (rows, columns)
    lower_bound: Minimum value for genes
    upper_bound: Maximum value for genes
    """
    rows, cols = grid_size
    return np.random.uniform(lower_bound, upper_bound, (rows, cols))

def evaluate_fitness(grid):
    """
    Evaluate the fitness of each cell in the grid.
    grid: 2D array representing the cellular grid
    """
    rows, cols = grid.shape
    fitness = np.zeros((rows, cols))
    for i in range(rows):
        for j in range(cols):
            fitness[i, j] = rastrigin_function([grid[i, j]])
    return fitness

def get_neighbors(grid, i, j):
    """
    Get the neighbors of a cell at position (i, j).
    grid: 2D array representing the grid
    i, j: Position of the cell
    """
    neighbors = []
    rows, cols = grid.shape
    for x in [-1, 0, 1]:
        for y in [-1, 0, 1]:
            if x == 0 and y == 0:
                continue
            ni, nj = (i + x) % rows, (j + y) % cols  # Wrap around the grid
            neighbors.append(grid[ni, nj])
    return neighbors

def update_cell_state(grid, i, j, lower_bound, upper_bound):
    """
    Update the state of a single cell based on its neighbors' fitness.
    grid: 2D array representing the grid
    i, j: Position of the cell
    lower_bound: Minimum value for genes
    upper_bound: Maximum value for genes
    """
    neighbors = get_neighbors(grid, i, j)
    if neighbors:
        new_value = np.mean(neighbors) + random.uniform(-0.1, 0.1)
        return np.clip(new_value, lower_bound, upper_bound)
    return grid[i, j]

def parallel_update(grid, lower_bound, upper_bound):
    """
    Perform a parallel update of the grid cells.
    grid: 2D array representing the grid
    lower_bound: Minimum value for genes
    upper_bound: Maximum value for genes
    """
    rows, cols = grid.shape
    new_grid = np.copy(grid)
    with ThreadPoolExecutor() as executor:
        futures = []
        for i in range(rows):
            for j in range(cols):
                futures.append(executor.submit(update_cell_state, grid, i, j, lower_bound, upper_bound))
        index = 0
        for i in range(rows):
            for j in range(cols):
                new_grid[i, j] = futures[index].result()
                index += 1
    return new_grid

def parallel_cellular_algorithm(grid_size, lower_bound, upper_bound, num_iterations):
    """
    Parallel Cellular Algorithm for optimization.
    grid_size: Tuple representing the grid dimensions (rows, columns)
    lower_bound: Minimum value for genes
    upper_bound: Maximum value for genes
    num_iterations: Number of iterations to perform
    """
    # Step 1: Initialize grid
    grid = initialize_population(grid_size, lower_bound, upper_bound)
    best_solution = None
    best_fitness = float('inf')

    for iteration in range(num_iterations):
        # Step 2: Evaluate fitness
        fitness = evaluate_fitness(grid)

        # Step 3: Track the best solution
        min_fitness = np.min(fitness)
        if min_fitness < best_fitness:
            best_fitness = min_fitness
            best_solution = grid[np.unravel_index(np.argmin(fitness), fitness.shape)]

        print(f"Iteration {iteration + 1}: Best Fitness = {best_fitness:.4f}")

        # Step 4: Update cell states in parallel
        grid = parallel_update(grid, lower_bound, upper_bound)

    print("Optimization Complete!")
    return best_solution, best_fitness

# Parameters
GRID_SIZE = (10, 10)  # Grid dimensions (rows x columns)
LOWER_BOUND = -5.12
UPPER_BOUND = 5.12
NUM_ITERATIONS = 5

# Run Parallel Cellular Algorithm
best_solution, best_fitness = parallel_cellular_algorithm(
    GRID_SIZE, LOWER_BOUND, UPPER_BOUND, NUM_ITERATIONS
)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)


Iteration 1: Best Fitness = 0.0344
Iteration 2: Best Fitness = 0.0095
Iteration 3: Best Fitness = 0.0095
Iteration 4: Best Fitness = 0.0001
Iteration 5: Best Fitness = 0.0001
Optimization Complete!
Best Solution: -0.0005231211286801374
Best Fitness: 5.429108011867356e-05
